In [6]:
import pandas as pd

In [9]:
df = pd.read_json(path_or_buf='gg2013.json')
df.head(10)

,id,text,timestamp_ms,user
0,290620657987887104,JLo's dress! #eredcarpet #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}"
1,290620657887219713,What's making Sofia Vergara's boobs stay like ...,2013-01-14 00:45:38,"{'screen_name': 'theAmberShow', 'id': 14648726}"
2,290620657828524032,RT @FabSugar: Kerry Washington is EVERYTHING. ...,2013-01-14 00:45:38,"{'screen_name': 'SweetyPW', 'id': 35498686}"
3,290620657799159809,Anne Hathaway has got me living. #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': '_NicoleEdwards', 'id': 144430..."
4,290620657778188288,Jennifer Lopez's lace dress? Thoughts? #Golden...,2013-01-14 00:45:38,"{'screen_name': 'lolaogunnaike', 'id': 134953223}"
5,290620657719455745,Podrán criticar a #Adele de su moda y su maniq...,2013-01-14 00:45:38,"{'screen_name': 'SabyOnford', 'id': 404554221}"
6,290620657715273728,"US, Weakly #GoldenGlobes",2013-01-14 00:45:38,"{'screen_name': 'alejandraoleary', 'id': 45748..."
7,290620657715253248,"RT @BillMc7: ""Wait. What's that smell?!"" (ever...",2013-01-14 00:45:38,"{'screen_name': 'xtinavickers', 'id': 92139052}"
8,290620657706872832,Hugh Jackman is so awesome!!! #goldenglobes,2013-01-14 00:45:38,"{'screen_name': 'Boaz_173', 'id': 31504713}"
9,290620657560084480,It was awkward. RT @hollywoodhwife: They cut t...,2013-01-14 00:45:38,"{'screen_name': 'KateSpencer1', 'id': 21571382}"


In [20]:
len(df)

174643

In [11]:
data = df['text']
data[:10]

0               JLo's dress! #eredcarpet #GoldenGlobes
1    What's making Sofia Vergara's boobs stay like ...
2    RT @FabSugar: Kerry Washington is EVERYTHING. ...
3       Anne Hathaway has got me living. #GoldenGlobes
4    Jennifer Lopez's lace dress? Thoughts? #Golden...
5    Podrán criticar a #Adele de su moda y su maniq...
6                             US, Weakly #GoldenGlobes
7    RT @BillMc7: "Wait. What's that smell?!" (ever...
8          Hugh Jackman is so awesome!!! #goldenglobes
9    It was awkward. RT @hollywoodhwife: They cut t...
Name: text, dtype: object

In [13]:
data[1]

"What's making Sofia Vergara's boobs stay like that? Magic? Witchcraft? #GoldenGlobes"

In [14]:
import re

In [52]:
def cleanse(line):
    # replace everything to ' ' except whitespace, alphanumeric character, apostrophe, hashtag, @
    return re.sub(r'[^\w\s\'#@]', ' ', line).lower()

In [53]:
# test
for line in data[:10]:
    print(line)
    print(cleanse(line))
    print(line.split())
    print()
    
#     break

JLo's dress! #eredcarpet #GoldenGlobes
jlo's dress  #eredcarpet #goldenglobes
["JLo's", 'dress!', '#eredcarpet', '#GoldenGlobes']

What's making Sofia Vergara's boobs stay like that? Magic? Witchcraft? #GoldenGlobes
what's making sofia vergara's boobs stay like that  magic  witchcraft  #goldenglobes
["What's", 'making', 'Sofia', "Vergara's", 'boobs', 'stay', 'like', 'that?', 'Magic?', 'Witchcraft?', '#GoldenGlobes']

RT @FabSugar: Kerry Washington is EVERYTHING. Dying over her Miu Miu gown! @goldenglobes #goldenglobes http://t.co/3feH7MvO
rt @fabsugar  kerry washington is everything  dying over her miu miu gown  @goldenglobes #goldenglobes http   t co 3feh7mvo
['RT', '@FabSugar:', 'Kerry', 'Washington', 'is', 'EVERYTHING.', 'Dying', 'over', 'her', 'Miu', 'Miu', 'gown!', '@goldenglobes', '#goldenglobes', 'http://t.co/3feH7MvO']

Anne Hathaway has got me living. #GoldenGlobes
anne hathaway has got me living  #goldenglobes
['Anne', 'Hathaway', 'has', 'got', 'me', 'living.', '#GoldenGlobes

In [ ]:
# to do
# compile/collect a list of nominees (actors/ actresses) and their possible name contractions/abrreviations
# research on the rules of Golden Globes nomination and voting mechanism

In [58]:
# test
# define patterns to match
pat = re.compile(r'jennifer lopez')

num = 0
for line in data:
    match = re.findall(pat, cleanse(line))
    if match:
        # print the first 10 occurrences
        if num < 10:
            print(match)
            print(line + '\n')
        num += 1

print(num)

['jennifer lopez']
Jennifer Lopez's lace dress? Thoughts? #GoldenGlobes

['jennifer lopez']
Jennifer Lopez's dress is jaw droppingly amazing #GoldenGlobes #redcarpet

['jennifer lopez']
.@kerrywashington's nude &amp; sheer = yes. Jennifer Lopez's take on the same trend? = oy vey #goldenglobes

['jennifer lopez']
JENNIFER LOPEZ LOOKS AMAZINGG!!!! @JLo #GoldenGlobes

['jennifer lopez']
Judging by her face, Jennifer Lopez is confusing the #GoldenGlobes Red Carpet with a Playboy photo shoot.

['jennifer lopez']
Jennifer Lopez. Shit. Damn. DAMN! #GoldenGlobes #redcarpet #iquit

['jennifer lopez']
Best dress of the night Jennifer Lopez...stunning #eredcarpet #goldenglobes

['jennifer lopez']
Tenho preguiça do Ben Affleck só pq ele já namorou a chatinha da Jennifer Lopez #GoldenGlobes

['jennifer lopez']
Jennifer Lopez just shut it down in nude lace. #goldenglobes

['jennifer lopez']
Jennifer Lopez looks AMAZING #GoldenGlobes @JLo

650
